In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import math
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from pprint import pprint
import pickle
import torch

# map district name to district number
district_mapping_path = "../airflow/dags/data/districts.xlsx"

district_mapping = pd.read_excel(district_mapping_path)


def get_district_name(district_no):
    return district_mapping[district_mapping["Postal District"] == district_no][
        "General Location"
    ].values[0]

# Import all previous files

In [12]:
# open pickle
train_df_dict = pd.read_pickle('data/train_df_dict.pkl')
train_df_dict_L = pd.read_pickle('data/train_df_dict_L.pkl')
test_df_dict = pd.read_pickle('data/test_df_dict.pkl')

# import resale_flat_transactions_clean
resale_flat_transactions_clean = pd.read_csv('data/resale_flat_transactions_clean.csv')

# import features 
all_district_var_ts = pd.read_pickle('data/all_district_var_ts.pkl')

# remove the NaN values from train set
# train_df_dict_clean = {}
# for district_no, district_df in train_df_dict.items():
#     train_df_dict_clean[district_no] = district_df.dropna()

# hopefully lstm can handle the NaN values


In [11]:
# prepare data for LSTM
def create_dataset(df_dict, district_no, look_back=1):
    dataX, dataY = [], []
    for i in range(len(df_dict[district_no]) - look_back - 1):
        a = df_dict[district_no][i : (i + look_back), 0]
        dataX.append(a)
        dataY.append(df_dict[district_no][i + look_back, 0])
    return np.array(dataX), np.array(dataY)